In [504]:
import math
import random
import matplotlib.pyplot as plt

In [505]:
def random_vector(minmax):
    vector = list()
    for i in range(len(minmax)):
        rand = minmax[i][0] + ((minmax[i][1] - minmax[i][0]) * random.random())
        vector.append(rand)
    return vector

In [506]:
def initialize_weights(num_weights):
    minmax = list()
    for i in range(num_weights):
        minmax.append([-random.random(), random.random()])
    return random_vector(minmax)

In [507]:
def activate(weights, vector):
    _sum = weights[-1] * 1.0  # I think this is the bias constant
    for i in range(len(vector)):
        _sum += weights[i] * vector[i]
    return _sum

In [532]:
def transfer(activation):
    return 1.0 * (1.0 + math.exp(-activation))

In [509]:
def transfer_derivative(output):
    return output * (1.0 - output)

In [510]:
def forward_propagate(network, vector):
    for i in range(len(network)):
        layer = network[i]
        _input = None
        if (i == 0):
            _input = vector
        else:
            new_vector = list()
            previous_layer = network[i - 1]
            for k in range(len(previous_layer)):
                new_vector.append(previous_layer[k]["output"])
            _input = new_vector
        for neuron in layer:
            neuron["activation"] = activate(neuron["weights"], _input)
            neuron["output"] = transfer(neuron["activation"])
    return network[-1][0]["output"]

In [511]:
def backward_propagate_error(network, expected_output):
    for i in range(len(network)):
        index = len(network) - 1 - i
        layer = network[index]
        if (index == (len(network) - 1)):
            neuron = layer[0] # assume one node in output layer
            error = (expected_output - neuron["output"])
            neuron["delta"] = error * transfer_derivative(neuron["output"])
        else:
            next_layer = network[index + 1]
            for j in range(len(layer)):
                err_sum = 0.0
                neuron = layer[j]
                for k in range(len(next_layer)):
                    next_neuron = next_layer[k]
                    err_sum += next_neuron["weights"][j] * next_neuron["delta"]
                neuron["delta"] = err_sum * transfer_derivative(neuron["output"])

In [512]:
def calculate_error_derivatives_for_weights(network, vector):
    for i in range(len(network)):
        layer = network[i]
        _input = None
        if (i == 0):
            _input = vector
        else:
            new_vector = list()
            previous_layer = network[i - 1]
            for k in range(len(previous_layer)):
                new_vector.append(previous_layer[k]["output"])
            _input = new_vector
        for neuron in layer:
            signal = None
            for k in range(len(_input)):
                signal = _input[k]
                neuron["deriv"][k] += neuron["delta"] * signal
            neuron["deriv"][-1] += neuron["delta"] * 1.0           

In [513]:
def update_weights(network, learning_rate, mom=0.8):
    for layer in network:
        for neuron in layer:
            for i in range(len(neuron["weights"])):
                delta = (learning_rate * neuron["deriv"][i]) + (neuron["last_delta"][i] * mom)
                neuron["weights"][i] += delta
                neuron["last_delta"][i] = delta
                neuron["deriv"][i] = 0.0

In [524]:
def train_network(network, domain, num_inputs, iterations, learning_rate):
    correct = 0
    for epoch in range(iterations):
        for pattern in domain:
            vector = list()
            for k in range(len(pattern)):
                vector.append(float(pattern[k]))
            expected = pattern[-1]
            output = forward_propagate(network, vector)
            if (round(output) == expected):
                correct += 1
            backward_propagate_error(network, expected)
            calculate_error_derivatives_for_weights(network, vector)
        update_weights(network, learning_rate)
        if (((epoch + 1) % 100) == 0):
            print("> epoch = " + str(epoch+1) + ", Correct = " + str(correct) + "/" + str(100 * len(domain)))
            print(network[0][0]["output"])
            correct = 0

In [525]:
def test_network(network, domain, num_inputs):
    correct = 0
    for pattern in domain:
        input_vector = list()
        for i in range(num_inputs):
            input_vector.append(float(pattern[i]))
        output = forward_propagate(network, input_vector)
        if (round(output) == pattern[-1]):
            correct += 1
    print("Finished test with a score of " + str(correct) + "/" + str(len(domain)))
    return correct

In [526]:
def create_neuron(num_inputs):
    neuron = {}
    neuron["weights"] = initialize_weights(num_inputs + 1)
    neuron["last_delta"] = [0.0] * (num_inputs + 1)
    neuron["deriv"] = [0.0] * (num_inputs + 1)
    return neuron

In [527]:
def execute(domain, num_inputs, iterations, num_nodes, learning_rate):
    network = [[]]
    for i in range(num_nodes):
        network[0].append(create_neuron(num_inputs))
    network.append([create_neuron(len(network[-1]))])
    print("Topology: inputs = " + str(num_inputs) + "  layers = " + str(len(network)))
    train_network(network, domain, num_inputs, iterations, learning_rate)
    test_network(network, domain, num_inputs)
    return network

In [531]:
if __name__ == "__main__":
    # problem configuration
    xor = [[0,0,0], [0,1,1], [1,0,1], [1,1,0]]
    inputs = 2
    # algorithm configuration
    learning_rate = 0.3
    num_hidden_nodes = 4
    iterations = 2000
    # execute the algorithm
    execute(xor, inputs, iterations, num_hidden_nodes, learning_rate)

Topology: inputs = 2  layers = 2
> epoch = 100, Correct = 200/400
2.478766614295832
> epoch = 200, Correct = 200/400
2.489924514307294
> epoch = 300, Correct = 200/400
2.500281279220596
> epoch = 400, Correct = 200/400
2.5099424147289344
> epoch = 500, Correct = 200/400
2.518993981453246
> epoch = 600, Correct = 200/400
2.527507084014929
> epoch = 700, Correct = 200/400
2.5355411372682184
> epoch = 800, Correct = 200/400
2.5431462884865463
> epoch = 900, Correct = 200/400
2.5503652441208007
> epoch = 1000, Correct = 200/400
2.557234668359711
> epoch = 1100, Correct = 200/400
2.5637862684145563
> epoch = 1200, Correct = 200/400
2.570047647031915
> epoch = 1300, Correct = 200/400
2.5760429796118345
> epoch = 1400, Correct = 200/400
2.5817935574724804
> epoch = 1500, Correct = 200/400
2.587318227768384
> epoch = 1600, Correct = 200/400
2.5926337527613272
> epoch = 1700, Correct = 200/400
2.597755105536531
> epoch = 1800, Correct = 200/400
2.602695715178128
> epoch = 1900, Correct = 200/40